In [22]:
import os 
import csv
import copy
import math 
import gzip
import torch
import random
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import torch.nn as nn
from sklearn import metrics
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
warnings.filterwarnings("ignore")

In [23]:
global device
device = 'cuda' if torch.cuda.is_available() else 'cpu' 

In [ ]:
def genome_data(data_file):
    data=open(data_file).read()
    chromosomes_data = data.split('>')[1:]
    return (chromosomes_data)

def Read_bed_file(chromosomes_data,peak_file,seq_length,Chr_dict):
    peaks = open(peak_file).readlines()
    if Chr_dict==None:
        Chr_dict = {}
        i=0
        for chrom_data in chromosomes_data:
            ref = chrom_data.split('\n')[0].split(' ')[-1][1:-1]
            Chr_dict[ref]=i
            i+=1
        Chr_dict['chromosome=Mito'] = Chr_dict.pop('top=circular')
    peak_sequences = []
    for peak in peaks:
        peak_split = peak.split('\t')
        Chr = 'chromosome='+str(peak_split[0])
        start_idx = max(int(peak_split[1])-seq_length//2,0)
        end_idx = min(int(len(chromosomes_data[Chr])), start_idx+seq_length)
        #end_idx = start_idx+seq_length
        if Chr in Chr_dict:
            peak_sequences.append(''.join(chromosomes_data[Chr_dict[Chr]].split('\n')[1:])[start_idx:end_idx])
    return (peak_sequences)

def Read_narrow_file(chromosomes_data,peak_file):
    peaks = open(peak_file).readlines()
    if Chr_dict==None:
        Chr_dict = {}
        i=0
        for chrom_data in chromosomes_data:
            ref = chrom_data.split('\n')[0].split(' ')[-1][1:-1]
            Chr_dict[ref]=i
            i+=1
        Chr_dict['chromosome=Mito'] = Chr_dict.pop('top=circular')
    for peak in peaks:
        peak_split = peak.split('\t')
        Chr = 'chromosome='+str(peak_split[0])
        start_idx = max(int(peak_split[1])-seq_length//2,0)
        end_idx = start_idx+seq_length
        if Chr in Chr_dict:
            peak_sequences.append(''.join(chromosomes_data[Chr_dict[Chr]].split('\n')[1:])[start_idx:end_idx])
    return (peak_sequences)
    return None

def dinucshuffle(sequence):
    b=[sequence[i:i+2] for i in range(0, len(sequence), 2)]
    random.shuffle(b)
    d=''.join([str(x) for x in b])
    return d

def seqtopad(sequence, motif_len):
    rows=len(sequence)+2*motif_len-2
    S=np.empty([rows,4])
    base=['A', 'C', 'G', 'T']
    for i in range(rows):
        for j in range(4):
            if (i-motif_len+1<len(sequence) and sequence[i-motif_len+1]=='N' 
                or i<motif_len-1 or i>len(sequence)+motif_len-2):
                S[i,j]=np.float32(0.25)
            elif sequence[i-motif_len+1]==base[j]:
                S[i,j]=np.float32(1)
            else:
                S[i,j]=np.float32(0)
    return np.transpose(S)

def extract_data(data_file,peak_file,motif_length=24,seq_length=150,Chr_dict=None):
    chromosomes_data = genome_data(data_file)
    if '.bed' in peak_file:
        peak_sequences = Read_bed_file(chromosomes_data,peak_file,seq_length,Chr_dict)
    else :
        peak_sequences = Read_narrow_file(chromosomes_data,peak_file,seq_length,Chr_dict)
    alldata = []
    for seq in peak_sequences:
        alldata.append([seqtopad(seq,motif_length),[1]])#
        alldata.append([seqtopad(dinucshuffle(seq),motif_length),[0]])#
    random.shuffle(alldata)
    size=int(len(alldata)/5)
    train_data=alldata[:4*size]
    valid_data=alldata[4*size:int(4.5*size)]
    test_data = alldata[int(4.5*size):]
    return train_data,valid_data,test_data,alldata,peak_sequences

In [ ]:
def Train_model(model,train_loader,valid_loader, l_rate=0.01 ,  maxepochs=100,epochs_for_early_stop=0,verbose=False):
    best_model = None
    best_loss = np.inf
    counter = 0
    nepochs=0
    valid_losses =[]
    train_losses = []
    optimizer = torch.optim.SGD(model.parameters(),lr=l_rate,momentum=0.9,nesterov=True,weight_decay=1e-02)
    criterion = nn.BCELoss(reduction='mean')
    while nepochs<maxepochs:
        model.train()
        train_loss=0
        for i, (data, target) in enumerate(train_loader):
            data = data.to(device)
            target = target.to(device)
            output = model(data)
            loss = criterion(output, target)#
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss+=loss.item()
        if verbose:
            print('Model trained for {0} epochs out of {2}. Training loss is {1}'.format(nepochs+1,loss.item(),maxepochs))
        train_losses.append(train_loss/(i+1))
        with torch.no_grad():
            model.eval() 
            valid_loss=0
            for i, (data, target) in enumerate(valid_loader):
                data = data.to(device)
                target = target.to(device)
                output = model(data)
                loss = F.binary_cross_entropy(output, target)#
                valid_loss+=loss.item()
            valid_losses.append(valid_loss/(i+1))
        counter+=1
        nepochs +=1
        if epochs_for_early_stop>0:
            if valid_losses[-1]<best_loss:
                if verbose:
                    print('Validation loss decreased from {0} to {1}'.format(best_loss,valid_losses[-1]))
                best_loss = valid_losses[-1]
                best_model = model
                counter = 0
            else:
                if verbose:
                    print('Counter for early stopping: {0} out of {1}'.format(counter,epochs_for_early_stop))
                if counter == epochs_for_early_stop:
                    print('early stopping at epoch ', nepochs-counter)
                    return (best_model,nepochs-counter,train_losses,valid_losses)
    print('no early stopping')
    return (best_model,nepochs,train_losses,valid_losses)

def Test_model(model,test_loader):
    with torch.no_grad():
        best_model.eval()
        pred_list = []
        labels_list = []
        for i, (data, target) in enumerate(test_loader):
            data = data.to(device)
            target = target.to(device)
            output = best_model(data)
            pred=output.cpu().detach().numpy().reshape(output.shape[0])
            labels=target.cpu().numpy().reshape(output.shape[0])
            pred_list.append(pred)
            labels_list.append(labels)
        labels = np.concatenate(labels_list)
        predictions = np.concatenate(pred_list)
        auc = metrics.roc_auc_score(labels, predictions)
        print('AUC on test data ', auc)
    return (auc)

In [ ]:
class dataset(Dataset):
    def __init__(self, xy=None):
        self.x_data=np.asarray([el[0] for el in xy],dtype=np.float32)
        self.y_data =np.asarray([el[1] for el in xy ],dtype=np.float32)
        self.x_data = torch.from_numpy(self.x_data)
        self.y_data = torch.from_numpy(self.y_data)
        self.length=len(self.x_data)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.length

In [ ]:
class Network(nn.Module):
    def __init__(self, num_motif , motif_len):
        super(Network, self).__init__()
        self.num_motif = num_motif
        self.conv = nn.Sequential(
            nn.Conv1d(4, num_motif, kernel_size=motif_len))
        self.classifier = nn.Sequential(
            nn.Linear(num_motif , num_motif),
            nn.ReLU(inplace=True),
            nn.Linear(num_motif, 1),
            nn.Sigmoid())
    def forward(self, x):
        x = self.conv(x)
        x,_ = torch.max(x, dim=2)
        predict = self.classifier(x)
        return predict
    

In [ ]:
data_file = 'Data/S288C_reference_sequence_R64-3-1_20210421.fsa'
peak_file = 'Data/Condensin_peaks_Log.bed'
seq_length = 150
motif_length = 24
train_data,valid_data,test_data,alldata,peak_sequences = extract_data(data_file,peak_file,motif_length,seq_length)
len(train_data),len(valid_data),len(test_data)

In [ ]:
train_dataset=dataset(train_data)
valid_dataset=dataset(valid_data)
test_dataset=dataset(test_data)
batch_size = 64
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,shuffle=False)
valid_loader = DataLoader(dataset=valid_dataset,
                          batch_size=batch_size,shuffle=False)
test_loader = DataLoader(dataset=test_dataset,
                          batch_size=batch_size,shuffle=False)

In [ ]:
verbose=True
num_motif=120
motif_len=24
sigma_conv=10**-3
sigma_w=0.1
num_conv_layers = 1
dropprob = 0.2
l_rate=0.001
maxepochs,epochs_for_early_stop = 500,50
model = Network(num_motif, motif_len).to(device)#num_conv_layers,dropprob
best_model,epochs,train_losses,valid_losses = Train_model(model,train_loader,valid_loader,l_rate ,maxepochs,epochs_for_early_stop,verbose)

In [ ]:
auc = Test_model(best_model,test_loader)